<a href="https://colab.research.google.com/github/SylviaNice/7162856-G-rez-Git-et-GitHub/blob/main/Projet_Bottleneck_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Python - Optimiser la gestion des données pour Bottleneck

In [45]:
# 2. Installation des bibliothèques

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from scipy.stats import zscore

In [46]:
# 1. Importation, chemins d'accès et chargement des fichiers

erp_df = 'https://docs.google.com/spreadsheets/d/1_bHNsOP3WqgVbWgTAtWv6C1qoMcgqUYL/edit?usp=share_link&ouid=104758612821007799222&rtpof=true&sd=true'
liaison_df = 'https://docs.google.com/spreadsheets/d/1QLx5-2sbes7bbnvmEKLiZdI8mdJM9y7s/edit?usp=share_link&ouid=104758612821007799222&rtpof=true&sd=true'
web_df = 'https://docs.google.com/spreadsheets/d/1ndp4EH_uGMu2x_8G_U6Tgby3g68NIKyV/edit?usp=share_link&ouid=104758612821007799222&rtpof=true&sd=true'

In [47]:
# 3. Aperçu des 3 fichiers importés

erp_df = pd.read_excel("Fichier_erp.xlsx")
web_df = pd.read_excel("Fichier_web.xlsx")
liaison_df = pd.read_excel("fichier_liaison.xlsx")
display(erp_df.head())
display(web_df.head())
display(liaison_df.head())

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,post_date_gmt,post_content,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,16004,0,0,0,0.0,5.0,NaN,NaN,2.0,2018-06-07 16:27:25,2018-06-07 14:27:25,NaN,Château du Couvent Pomerol 2017,NaN,publish,closed,closed,NaN,chateau-du-couvent-pomerol-2017,2020-08-25 18:35:02,2020-08-25 16:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15075,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-14 15:39:43,2018-02-14 14:39:43,NaN,Parés Baltà Penedès Indigena 2017,Des couleurs et aromes intenses où le fruit et...,publish,closed,closed,NaN,pares-balta-penedes-indigena-2017,2020-08-20 15:35:02,2020-08-20 13:35:02,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,16209,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-14 17:15:31,2018-02-14 16:15:31,NaN,Maurel Cabardès Tradition 2017,"Un joli nez aux arômes de fruits rouges, de ca...",publish,closed,closed,NaN,maurel-cabardes-tradition-2017,2020-08-05 18:05:03,2020-08-05 16:05:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,15763,0,0,0,0.0,1.0,NaN,NaN,2.0,2020-01-25 14:08:16,2020-01-25 13:08:16,NaN,Domaine de la Monardière Vacqueyras Vieilles V...,NaN,publish,closed,closed,NaN,domaine-de-la-monardiere-vacqueyras-les-vieill...,2020-08-21 11:35:02,2020-08-21 09:35:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0


,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


In [54]:
# Nombre de lignes et de colonnes de chacun des fichiers

print(erp_df.shape)
print(web_df.shape)
print(liaison_df.shape)

(825, 5)
(1513, 28)
(825, 2)


In [56]:
# Afficher nom et type de chaque colonne pour erp
print("🗂️ Colonnes de erp")
print(erp_df.dtypes)
print("-" * 40)

# Afficher nom et type de chaque colonne pour web
print("🗂️ Colonnes de web")
print(web_df.dtypes)
print("-" * 40)

# Afficher nom et type de chaque colonne pour liaison
print("🗂️ Colonnes de liaison")
print(liaison_df.dtypes)
print("-" * 40)

🗂️ Colonnes de erp
product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
dtype: object
----------------------------------------
🗂️ Colonnes de web
sku                              object
virtual                           int64
downloadable                      int64
rating_count                      int64
average_rating                  float64
total_sales                     float64
tax_status                       object
tax_class                       float64
post_author                     float64
post_date                datetime64[ns]
post_date_gmt            datetime64[ns]
post_content                    float64
post_title                       object
post_excerpt                     object
post_status                      object
comment_status                   object
ping_status                      object
post_password                   float64
post_name                        object
post_modified    

In [64]:
# Renommer la colonne (rectification de l'erreur de la stagiaire) et fusionner les 3 fichiers de données

liaison_df = pd.read_excel("fichier_liaison.xlsx")
liaison_df = liaison_df.rename(columns={'id_web': 'SKU'})
print(liaison_df)

     product_id      SKU
0          3847    15298
1          3849    15296
2          3850    15300
3          4032    19814
4          4039    19815
..          ...      ...
820        7203      NaN
821        7204      NaN
822        7247  13127-1
823        7329  14680-1
824        7338    16230

[825 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
